In [1]:
import tensorflow as tf
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt

In [21]:
#biomag_labeled_1 = sio.loadmat('data_labeled_5.mat')
biomag_labeled_1 = sio.loadmat(r"D:\python_project\wip\data_labeled_for_py_1_2.mat")

trX, trY, teX, teY, vaX, vaY = biomag_labeled_1['x_train'], biomag_labeled_1['y_train'],\
                               biomag_labeled_1['y_test'], biomag_labeled_1['x_test'],\
                               biomag_labeled_1['x_val'], biomag_labeled_1['y_val'] 

epochs = 50
alpha = 1.0
batchsize = 100

In [22]:
sio.whosmat(r"D:\python_project\wip\data_labeled_for_py_1_2.mat")

[('x_train', (2094, 254), 'double'),
 ('y_train', (1, 2094), 'double'),
 ('x_test', (206, 254), 'double'),
 ('y_test', (206, 1), 'uint8'),
 ('x_val', (317, 254), 'double'),
 ('y_val', (317, 12), 'uint8')]

In [23]:
trY=np.reshape(trY,2094)
trY.shape

(2094,)

In [4]:
X = tf.placeholder("float", [None, 254])
Y = tf.placeholder("float", [None, 12])

rbm_w = tf.placeholder("float", [254, 500])
rbm_vb = tf.placeholder("float", [254])
rbm_hb = tf.placeholder("float", [500])

h0 = tf.nn.sigmoid(tf.matmul(X, rbm_w) + rbm_hb)
v1 = tf.nn.sigmoid(tf.matmul(h0, tf.transpose(rbm_w)) + rbm_vb)
h1 = tf.nn.sigmoid(tf.matmul(v1, rbm_w) + rbm_hb)

In [5]:
#grad calculation
w_positive_grad = tf.matmul(tf.transpose(X), h0)
w_negative_grad = tf.matmul(tf.transpose(v1), h1)

update_w = rbm_w + alpha * (w_positive_grad - w_negative_grad) / tf.to_float(tf.shape(X)[0])
update_vb = rbm_vb + alpha * tf.reduce_mean(X - v1, 0)
update_hb = rbm_hb + alpha * tf.reduce_mean(h0 - h1, 0)

h_sample = tf.nn.sigmoid(tf.matmul(X, rbm_w) + rbm_hb)
v_sample = tf.nn.sigmoid(tf.matmul(h_sample, tf.transpose(rbm_w)) + rbm_vb)

#define MSE
err = X - v_sample
err_sum = tf.reduce_mean(err * err)

sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [7]:
#init weights and biases
n_w = np.zeros([254, 500], np.float32)
n_vb = np.zeros([254], np.float32)
n_hb = np.zeros([500], np.float32)
o_w = np.zeros([254, 500], np.float32)
o_vb = np.zeros([254], np.float32)
o_hb = np.zeros([500], np.float32)

#training
for e in range(epochs):
    print(sess.run(err_sum, feed_dict={X: trX, rbm_w: o_w, rbm_vb: o_vb, rbm_hb: o_hb}))
    for start, end in zip(range(0, len(trX), batchsize), range(batchsize, len(trX), batchsize)):
        batch = trX[start:end]
        n_w = sess.run(update_w, feed_dict={X: batch, rbm_w: o_w, rbm_vb: o_vb, rbm_hb: o_hb})
        n_vb = sess.run(update_vb, feed_dict={X: batch, rbm_w: o_w, rbm_vb: o_vb, rbm_hb: o_hb})
        n_hb = sess.run(update_hb, feed_dict={X: batch, rbm_w: o_w, rbm_vb: o_vb, rbm_hb: o_hb})
        o_w = n_w
        o_vb = n_vb
        o_hb = n_hb
        if start % 2000 == 0:
            print("MSE: ", sess.run(err_sum, feed_dict={X: trX, rbm_w: n_w, rbm_vb: n_vb, rbm_hb: n_hb}))


0.124251075
MSE:  0.074030735


0.018301835
MSE:  0.018232886
0.01798589


MSE:  0.017976513
0.017960126
MSE:  0.017953912


0.0179548
MSE:  0.017949054
0.017953089
MSE:  0.017947452
0.017952353


MSE:  0.017946748
0.017951949
MSE:  0.017946353
0.017951684
MSE:  0.017946092


...

0.017949456
MSE:  0.01794382
0.017949414
MSE:  0.017943777
0.01794937
MSE:  0.017943736


0.017949332
MSE:  0.017943693
0.01794929
MSE:  0.017943652
0.01794925


MSE:  0.017943608
0.017949207
MSE:  0.017943565
0.017949166
MSE:  0.017943522


0.017949123
MSE:  0.017943477
0.017949082
MSE:  0.017943434
0.017949037
MSE:  0.01794339


0.01794899
MSE:  0.017943345


#2nd layer
---------------------------------------------------------------------------------------------

In [8]:
rbm_w2 = tf.placeholder("float", [500, 500])
rbm_vb2 = tf.placeholder("float", [500])
rbm_hb2 = tf.placeholder("float", [500])

h2 = tf.nn.sigmoid(tf.matmul(h0, rbm_w2) + rbm_hb2)
v2 = tf.nn.sigmoid(tf.matmul(h2, tf.transpose(rbm_w2)) + rbm_vb2)
h3 = tf.nn.sigmoid(tf.matmul(v2, rbm_w2) + rbm_hb2)

#grad calculation
w_positive_grad = tf.matmul(tf.transpose(h0), h2)
w_negative_grad = tf.matmul(tf.transpose(v2), h3)

update_w = rbm_w2 + alpha * (w_positive_grad - w_negative_grad) / tf.to_float(tf.shape(h1)[0])
update_vb = rbm_vb2 + alpha * tf.reduce_mean(h0 - v2, 0)
update_hb = rbm_hb2 + alpha * tf.reduce_mean(h2 - h3, 0)

h_sample = tf.nn.sigmoid(tf.matmul(h0, rbm_w2) + rbm_hb2)
v_sample = tf.nn.sigmoid(tf.matmul(h_sample, tf.transpose(rbm_w2)) + rbm_vb2)

#define MSE
#X = tf.placeholder("float", [None, 784])          #error dimension fix  -s -k  | X-et kivenni
err = h0 - v_sample
err_sum = tf.reduce_mean(err * err)

sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

n_w_2 = np.zeros([500, 500], np.float32)
n_vb_2 = np.zeros([500], np.float32)
n_hb_2 = np.zeros([500], np.float32)
o_w_2 = np.zeros([500, 500], np.float32)
o_vb_2 = np.zeros([500], np.float32)
o_hb_2 = np.zeros([500], np.float32)


#training
print(sess.run(err_sum, feed_dict={rbm_w2: o_w_2, rbm_vb2: o_vb_2, rbm_hb2: o_hb_2, X: trX, rbm_hb : o_hb, rbm_w: o_w}))
for start, end in zip(range(0, len(trX), batchsize), range(batchsize, len(trX), batchsize)):
    batch = trX[start:end]
    n_w_2 = sess.run(update_w, feed_dict={X: batch, rbm_w2: o_w_2, rbm_vb2: o_vb_2, rbm_hb2: o_hb_2,  rbm_hb : o_hb, rbm_w: o_w,
                                          rbm_vb: o_vb})
    n_vb_2 = sess.run(update_vb, feed_dict={X: batch, rbm_w2: o_w_2, rbm_vb2: o_vb_2, rbm_hb2: o_hb_2,  rbm_hb : o_hb, rbm_w: o_w,
                                            rbm_vb: o_vb})
    n_hb_2 = sess.run(update_hb, feed_dict={X: batch, rbm_w2: o_w_2, rbm_vb2: o_vb_2, rbm_hb2: o_hb_2,  rbm_hb : o_hb, rbm_w: o_w,
                                            rbm_vb: o_vb})
    o_w_2 = n_w_2
    o_vb_2 = n_vb_2
    o_hb_2 = n_hb_2
    if start % 1000 == 0:
        print("MSE: ", sess.run(err_sum, feed_dict={rbm_w2: o_w_2, rbm_vb2: o_vb_2, rbm_hb2: o_hb_2, X: trX, rbm_hb : o_hb, rbm_w: o_w}))



0.24987753
MSE:  6.3008365e-08
MSE:  6.3008365e-08


#3nd layer
---------------------------------------------------------------------------------------------

In [9]:
rbm_w3 = tf.placeholder("float", [500, 500])
rbm_vb3 = tf.placeholder("float", [500])
rbm_hb3 = tf.placeholder("float", [500])

h4 = tf.nn.sigmoid(tf.matmul(h2, rbm_w3) + rbm_hb3)
v3 = tf.nn.sigmoid(tf.matmul(h4, tf.transpose(rbm_w3)) + rbm_vb3)
h5 = tf.nn.sigmoid(tf.matmul(v3, rbm_w3) + rbm_hb3)

#grad calculation
w_positive_grad = tf.matmul(tf.transpose(h2), h4)
w_negative_grad = tf.matmul(tf.transpose(v3), h5)

update_w = rbm_w3 + alpha * (w_positive_grad - w_negative_grad) / tf.to_float(tf.shape(h2)[0])
update_vb = rbm_vb3 + alpha * tf.reduce_mean(h2 - v3, 0)
update_hb = rbm_hb3 + alpha * tf.reduce_mean(h4 - h5, 0)

h_sample = tf.nn.sigmoid(tf.matmul(h2, rbm_w3) + rbm_hb3)
v_sample = tf.nn.sigmoid(tf.matmul(h_sample, tf.transpose(rbm_w3)) + rbm_vb3)

#define MSE
#X = tf.placeholder("float", [None, 784])          #error dimension fix  -s -k  | X-et kivenni
err = h2 - v_sample
err_sum = tf.reduce_mean(err * err)

sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

n_w_3 = np.zeros([500, 500], np.float32)
n_vb_3 = np.zeros([500], np.float32)
n_hb_3 = np.zeros([500], np.float32)
o_w_3 = np.zeros([500, 500], np.float32)
o_vb_3 = np.zeros([500], np.float32)
o_hb_3 = np.zeros([500], np.float32)


#training
print(sess.run(err_sum, feed_dict={rbm_w3: o_w_3, rbm_vb3: o_vb_3, rbm_hb3: o_hb_3, X: trX,
                                   rbm_hb2 : o_hb_2, rbm_w2: o_w_2,
                                   rbm_hb : o_hb, rbm_w: o_w, rbm_vb: o_vb}))
for start, end in zip(range(0, len(trX), batchsize), range(batchsize, len(trX), batchsize)):
    batch = trX[start:end]
    n_w_3 = sess.run(update_w, feed_dict={X: batch, rbm_w3: o_w_3, rbm_vb3: o_vb_3, rbm_hb3: o_hb_3,
                                          rbm_w2: o_w_2, rbm_vb2: o_vb_2, rbm_hb2: o_hb_2,
                                          rbm_hb : o_hb, rbm_w: o_w, rbm_vb: o_vb})
    n_vb_3 = sess.run(update_vb, feed_dict={X: batch, rbm_w3: o_w_3, rbm_vb3: o_vb_3, rbm_hb3: o_hb_3,
                                          rbm_w2: o_w_2, rbm_vb2: o_vb_2, rbm_hb2: o_hb_2,
                                          rbm_hb : o_hb, rbm_w: o_w, rbm_vb: o_vb})
    n_hb_3 = sess.run(update_hb, feed_dict={X: batch, rbm_w3: o_w_3, rbm_vb3: o_vb_3, rbm_hb3: o_hb_3,
                                          rbm_w2: o_w_2, rbm_vb2: o_vb_2, rbm_hb2: o_hb_2,
                                          rbm_hb : o_hb, rbm_w: o_w, rbm_vb: o_vb})
    o_w_3 = n_w_3
    o_vb_3 = n_vb_3
    o_hb_3 = n_hb_3
    if start % 1000 == 0:
        print("MSE: ", sess.run(err_sum, feed_dict={rbm_w3: o_w_3, rbm_vb3: o_vb_3, rbm_hb3: o_hb_3, X: trX,
                                                    rbm_w2: o_w_2, rbm_vb2: o_vb_2, rbm_hb2: o_hb_2,
                                                    rbm_hb : o_hb, rbm_w: o_w}))



0.0005170225
MSE:  0.004894533
MSE:  1.5767986e-05


In [26]:
#Classification---------------------------------------------------------------------------------------------
#weigths and biases for new output layer
out_w = tf.Variable(tf.random_normal([500, 12]))
out_b = tf.Variable(tf.random_normal([12]))
learning_rate = 0.01

# #reformat y train labels
trY_formazott = np.zeros([biomag_labeled_1['y_train'].shape[1],12])
# #biomag_labeled_1['y_test']
for rows in range((biomag_labeled_1['y_train'].shape[1])):
    #rows +=1
    current_class_label = biomag_labeled_1['y_train'][0,rows] 
    current_class_label = int(current_class_label)
    trY_formazott[rows,current_class_label -1] = 1
    # if biomag_labeled_1['y_train'][rows] != 0:
    #     trY_formazott[rows] = columns+1 #trY_formazando[rows,columns]
trY = trY_formazott




In [29]:
#trY[567]
trY.shape
#trY_formazott[4]

(2094, 12)

In [33]:
#network definition
def neural_net(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.nn.sigmoid(tf.matmul(x, o_w) + o_hb)
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_1, out_w) + out_b
    return out_layer

# Construct model
logits = neural_net(X)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init2 = tf.global_variables_initializer()
# Run the initializer
sess.run(init2)




#biomag_labeled_1['y_test']
#Y
#trY[355]
#biomag_labeled_1['y_train'][2,0]
#trY_formazott = np.zeros([biomag_labeled_1['y_train'].shape[0],12])
#Y
#trY_formazott[0]

In [43]:
#Training top ANN layer
for epochs_2 in range(30):
    for start, end in zip(range(0, len(trX), batchsize), range(batchsize, len(trX), batchsize)):
        batch_x = trX[start:end]
        batch_y = trY[start:end]
        
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if start % 1000 == 0:
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x, Y: batch_y})
            print("Step " + str(start/batchsize) + ", Minibatch Loss= " + \
              "{:.4f}".format(loss) + ", Training Accuracy= " + \
              "{:.3f}".format(acc))


#batch_x.shape
#trY_formazott.shape

Step 0.0, Minibatch Loss= 1.7293, Training Accuracy= 0.480
Step 10.0, Minibatch Loss= 1.8501, Training Accuracy= 0.460
Step 0.0, Minibatch Loss= 1.7293, Training Accuracy= 0.480
Step 10.0, Minibatch Loss= 1.8500, Training Accuracy= 0.460
Step 0.0, Minibatch Loss= 1.7292, Training Accuracy= 0.480
Step 10.0, Minibatch Loss= 1.8499, Training Accuracy= 0.460
Step 0.0, Minibatch Loss= 1.7291, Training Accuracy= 0.480
Step 10.0, Minibatch Loss= 1.8498, Training Accuracy= 0.460
Step 0.0, Minibatch Loss= 1.7291, Training Accuracy= 0.480
Step 10.0, Minibatch Loss= 1.8496, Training Accuracy= 0.460
Step 0.0, Minibatch Loss= 1.7290, Training Accuracy= 0.480
Step 10.0, Minibatch Loss= 1.8495, Training Accuracy= 0.460
Step 0.0, Minibatch Loss= 1.7289, Training Accuracy= 0.480


Step 10.0, Minibatch Loss= 1.8494, Training Accuracy= 0.460
Step 0.0, Minibatch Loss= 1.7289, Training Accuracy= 0.480
Step 10.0, Minibatch Loss= 1.8493, Training Accuracy= 0.460
Step 0.0, Minibatch Loss= 1.7288, Training Accuracy= 0.480
Step 10.0, Minibatch Loss= 1.8492, Training Accuracy= 0.460
Step 0.0, Minibatch Loss= 1.7288, Training Accuracy= 0.480
Step 10.0, Minibatch Loss= 1.8491, Training Accuracy= 0.460
Step 0.0, Minibatch Loss= 1.7287, Training Accuracy= 0.480


Step 10.0, Minibatch Loss= 1.8490, Training Accuracy= 0.460
Step 0.0, Minibatch Loss= 1.7286, Training Accuracy= 0.480
Step 10.0, Minibatch Loss= 1.8489, Training Accuracy= 0.460
Step 0.0, Minibatch Loss= 1.7286, Training Accuracy= 0.480
Step 10.0, Minibatch Loss= 1.8488, Training Accuracy= 0.460


Step 0.0, Minibatch Loss= 1.7285, Training Accuracy= 0.480
Step 10.0, Minibatch Loss= 1.8487, Training Accuracy= 0.460
Step 0.0, Minibatch Loss= 1.7285, Training Accuracy= 0.480

Step 10.0, Minibatch Loss= 1.8486, Training Accuracy= 0.460
Step 0.0, Minibatch Loss= 1.7284, Training Accuracy= 0.480
Step 10.0, Minibatch Loss= 1.8485, Training Accuracy= 0.460
Step 0.0, Minibatch Loss= 1.7284, Training Accuracy= 0.480
Step 10.0, Minibatch Loss= 1.8483, Training Accuracy= 0.460
Step 0.0, Minibatch Loss= 1.7283, Training Accuracy= 0.480
Step 10.0, Minibatch Loss= 1.8482, Training Accuracy= 0.460
Step 0.0, Minibatch Loss= 1.7283, Training Accuracy= 0.470
Step 10.0, Minibatch Loss= 1.8481, Training Accuracy= 0.460


Step 0.0, Minibatch Loss= 1.7282, Training Accuracy= 0.470
Step 10.0, Minibatch Loss= 1.8481, Training Accuracy= 0.460
Step 0.0, Minibatch Loss= 1.7282, Training Accuracy= 0.470


Step 10.0, Minibatch Loss= 1.8480, Training Accuracy= 0.460
Step 0.0, Minibatch Loss= 1.7281, Training Accuracy= 0.470
Step 10.0, Minibatch Loss= 1.8479, Training Accuracy= 0.460
Step 0.0, Minibatch Loss= 1.7281, Training Accuracy= 0.470
Step 10.0, Minibatch Loss= 1.8478, Training Accuracy= 0.460
Step 0.0, Minibatch Loss= 1.7280, Training Accuracy= 0.470


Step 10.0, Minibatch Loss= 1.8477, Training Accuracy= 0.460
Step 0.0, Minibatch Loss= 1.7280, Training Accuracy= 0.470
Step 10.0, Minibatch Loss= 1.8476, Training Accuracy= 0.460
Step 0.0, Minibatch Loss= 1.7279, Training Accuracy= 0.470
Step 10.0, Minibatch Loss= 1.8475, Training Accuracy= 0.460
Step 0.0, Minibatch Loss= 1.7279, Training Accuracy= 0.470
Step 10.0, Minibatch Loss= 1.8474, Training Accuracy= 0.460
Step 0.0, Minibatch Loss= 1.7278, Training Accuracy= 0.470
Step 10.0, Minibatch Loss= 1.8473, Training Accuracy= 0.460


Step 0.0, Minibatch Loss= 1.7278, Training Accuracy= 0.470
Step 10.0, Minibatch Loss= 1.8472, Training Accuracy= 0.460
Step 0.0, Minibatch Loss= 1.7277, Training Accuracy= 0.470
Step 10.0, Minibatch Loss= 1.8471, Training Accuracy= 0.460


In [36]:
#biomag_labeled_1['y_train'].shape[0]
# Evaluate the Model
# Define the input function for evaluating
input_fn_test = tf.estimator.inputs.numpy_input_fn(
    x={'val_data_x': vaX}, y=vaY,
    batch_size=batchsize, shuffle=False)
# Use the Estimator 'evaluate' method
model.evaluate(input_fn_test)





NameError: name 'model' is not defined

In [38]:
correct_pred

<tf.Tensor 'Equal_1:0' shape=(?,) dtype=bool>

In [82]:
# predictions = np.zeros(25)
# i = 0
true_preds = 0

for start, end in zip(range(0, len(trX), batchsize), range(batchsize, len(trX), batchsize)):
    batch_x = trX[start:end]
    batch_y = trY[start:end]
    pred = sess.run(correct_pred, feed_dict={X: batch_x, Y: batch_y})
    
    for i in range(len(pred)):
        if pred[i] == True:
            true_preds = true_preds + 1
    # predictions[i] = pred
    # i = i + 1
    
    
    #predictions[i] = pred
    


In [83]:
# true_preds = 0
# for i in range(len(pred)):
#     if pred[i] == True:
#         true_preds = true_preds + 1
        
#true_preds = true_preds
print(true_preds, '/',len(pred))
        

1002 / 100


In [77]:
predictions = true_preds
predictions

51

In [79]:
pred.shape

(100,)